In [7]:
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from os import path

import konlpy
from konlpy.tag import Okt

import gensim
from gensim import corpora, models
from gensim.models import Phrases
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary, MmCorpus

import pyLDAvis
import pyLDAvis.gensim

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource

In [9]:
indi = pd.read_csv('songs_rawdata\인디_5000곡_240402.csv')
indi = indi.iloc[:, 1:]
print(len(indi))
indi.head()

5000


,Title,Likes,Singer,Lyrics
0,인사,"105,245",범진,돌아서는 너를 보며\r\n난 아무 말도 할 수 없었고\r\n슬퍼하기엔 짧았던\r\n...
1,주저하는 연인들을 위해,"415,001",잔나비,나는 읽기 쉬운 마음이야\r\n당신도 스윽 훑고 가셔요\r\n달랠 길 없는 외로운 ...
2,사랑할 수밖에,"35,768",볼빨간사춘기,간지러웠어 널 보는 내 손이\r\n다 주고 싶었어 내 안에 남은 사랑까지\r\n지켜...
3,그대가 내 안에 박혔다(그내박),"38,463",순순희(기태),한참을 하늘을 보고 걸어 갔어\r\n내 지난 사랑은 온통 미련으로 가득한데\r\n사...
4,해요 (2022),"97,213",#안녕,그녀와 나는요 그땐 참 어렸어요\r\n많이 사랑했고 때론 많이 다퉜었죠\r\n지금 ...


In [10]:
%%time
text_filepath = 'indi_text.txt'
with open(text_filepath, 'w', encoding = 'utf-8') as f:
    for lyrics in indi.Lyrics.values:
        if pd.isnull(lyrics):
            continue
        f.write(lyrics+'\n')

CPU times: total: 15.6 ms
Wall time: 30.8 ms


In [11]:
bigram_model_filepath = 'bigram_model'
sentences_normalized_path = 'indi_text.txt'
unigram_sentences = LineSentence(sentences_normalized_path)
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)

In [12]:
%%time
bigram_sentences_filepath = 'sentences_for_word2vec.txt'
with open(bigram_sentences_filepath, 'w', encoding = 'utf-8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = bigram_model[unigram_sentence]
        f.write(' '.join(bigram_sentence) + '\n')

CPU times: total: 1.62 s
Wall time: 1.69 s


In [20]:
%%time
all2vec_filepath = 'all_word2vec_model'

lyrics_for_word2vec = LineSentence(bigram_sentences_filepath)
all2vec = Word2Vec(lyrics_for_word2vec, vector_size = 100, window = 5, min_count = 1, sg = 1)
all2vec.train(lyrics_for_word2vec, total_examples = 210235, epochs = 10)
all2vec.save(all2vec_filepath)

CPU times: total: 1min 11s
Wall time: 1min 2s


In [21]:
all2vec.wv.vectors.shape

(82039, 100)

In [22]:
print(all2vec.wv.most_similar('사랑'))

[('영원', 0.6139925718307495), ('내_고향', 0.5941444039344788), ('지금까지_받은', 0.5818619728088379), ('그것은', 0.5766550302505493), ('고백이라는', 0.5731273293495178), ('안녕_잘', 0.5722740888595581), ('인해', 0.5674343109130859), ('믿음', 0.5645585656166077), ('지질한', 0.5642980933189392), ('그것이', 0.5617907047271729)]


In [23]:
print(all2vec.wv.most_similar('첫번째'))

[('속내', 0.9550658464431763), ('구멍도', 0.9548126459121704), ('슬픔들', 0.9534212946891785), ('버릇과', 0.9522430896759033), ('옷자락', 0.9521971344947815), ('담그고', 0.9500828385353088), ('항기가', 0.948543906211853), ('아버지보다', 0.9484353065490723), ('자그마한,', 0.9480425119400024), ('내마음', 0.9467955231666565)]


In [24]:
all2vec_1 = Word2Vec.load('all_word2vec_model')

In [29]:
print(all2vec_1.wv.most_similar(positive = ['사랑'], topn = 5))
print()
print(all2vec_1.wv.most_similar(negative = ['사랑'], topn = 5))

[('영원', 0.6139925718307495), ('내_고향', 0.5941444039344788), ('지금까지_받은', 0.5818619728088379), ('그것은', 0.5766550302505493), ('고백이라는', 0.5731273293495178)]

[('헤어지지말자', 0.3514772653579712), ('Ringing', 0.2806505560874939), ('뚜루루르', 0.2667856514453888), ('april_funk!', 0.26600775122642517), ('꿈속뿐이네', 0.26255476474761963)]
